# News Modeling

Topic modeling involves **extracting features from document terms** and using
mathematical structures and frameworks like matrix factorization and SVD to generate **clusters or groups of terms** that are distinguishable from each other and these clusters of words form topics or concepts

Topic modeling is a method for **unsupervised classification** of documents, similar to clustering on numeric data

These concepts can be used to interpret the main **themes** of a corpus and also make **semantic connections among words that co-occur together** frequently in various documents

Topic modeling can help in the following areas:
- discovering the **hidden themes** in the collection
- **classifying** the documents into the discovered themes
- using the classification to **organize/summarize/search** the documents

Frameworks and algorithms to build topic models:
- Latent semantic indexing
- Latent Dirichlet allocation
- Non-negative matrix factorization

## Latent Dirichlet Allocation (LDA)
The latent Dirichlet allocation (LDA) technique is a **generative probabilistic model** where each **document is assumed to have a combination of topics** similar to a probabilistic latent semantic indexing model

In simple words, the idea behind LDA is that of two folds:
- each **document** can be described by a **distribution of topics**
- each **topic** can be described by a **distribution of words**

### LDA Algorithm

- 1. For each document, **randomly initialize each word to one of the K topics** (k is chosen beforehand)
- 2. For each document D, go through each word w and compute:
    - **P(T |D)** , which is a proportion of words in D assigned to topic T
    - **P(W |T )** , which is a proportion of assignments to topic T over all documents having the word W
- **Reassign word W with topic T** with probability P(T |D)´ P(W |T ) considering all other words and their topic assignments

![LDA](https://raw.githubusercontent.com/subashgandyer/datasets/main/images/LDA.png)

### Steps
- Install the necessary library
- Import the necessary libraries
- Download the dataset
- Load the dataset
- Pre-process the dataset
    - Stop words removal
    - Email removal
    - Non-alphabetic words removal
    - Tokenize
    - Lowercase
    - BiGrams & TriGrams
    - Lemmatization
- Create a dictionary for the document
- Filter low frequency words
- Create an Index to word dictionary
- Train the Topic Model
- Predict on the dataset
- Evaluate the Topic Model
    - Model Perplexity
    - Topic Coherence
- Visualize the topics

### Install the necessary library

In [261]:
#! pip install pyLDAvis gensim spacy

### Import the libraries

In [262]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from pprint import pprint
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim import corpora, models
import gensim
import json

### Download the dataset
Dataset: https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

#### 20-Newsgroups dataset
- 11K newsgroups posts
- 20 news topics

In [263]:
#!wget https://raw.githubusercontent.com/subashgandyer/datasets/main/newsgroups.json

### Load the dataset

In [264]:
f = open('/Users/jjimenez/Documents/Estudio/College/George Brown/Semester I/ML-II/GBC-ML-II/task2/newsgroups.json')
data = json.load(f)

In [265]:
data = str(data['content'])[1:100000]
print(data)

'0': "From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n", '1': "From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\

### Preprocess the data

### Email Removal

In [266]:
import re

e = '\S*@\S*\s?'
pattern = re.compile(e)
data = pattern.sub('', data)
print(pattern.sub('', data))


'0': "From: (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n", '1': "From: (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington

<>:3: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \S
<>:3: DeprecationWarning: invalid escape sequence \S
<ipython-input-266-0dbb1414ca5e>:3: DeprecationWarning: invalid escape sequence \S
  e = '\S*@\S*\s?'


### Newline Removal

In [267]:
data = data.replace("\\n", "")
data = data.replace("\\", "")
print(data)

'0': "From: (where's my thing)Subject: WHAT car is this!?Nntp-Posting-Host: rac3.wam.umd.eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day. It was a 2-door sports car, looked to be from the late 60s/early 70s. It was called a Bricklin. The doors were really small. In addition,the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail.Thanks,- IL   ---- brought to you by your neighborhood Lerxst ----", '1': "From: (Guy Kuo)Subject: SI Clock Poll - Final CallSummary: Final call for SI clock reportsKeywords: SI,acceleration,clock,upgradeArticle-I.D.: shelley.1qvfo9INNc3sOrganization: University of WashingtonLines: 11NNTP-Posting-Host: carson.u.washington.eduA fair number of brave souls who upgraded their SI c

### Single Quotes Removal

In [268]:
data = data.replace("'", "")
print(data)

0: "From: (wheres my thing)Subject: WHAT car is this!?Nntp-Posting-Host: rac3.wam.umd.eduOrganization: University of Maryland, College ParkLines: 15 I was wondering if anyone out there could enlighten me on this car I sawthe other day. It was a 2-door sports car, looked to be from the late 60s/early 70s. It was called a Bricklin. The doors were really small. In addition,the front bumper was separate from the rest of the body. This is all I know. If anyone can tellme a model name, engine specs, yearsof production, where this car is made, history, or whatever info youhave on this funky looking car, please e-mail.Thanks,- IL   ---- brought to you by your neighborhood Lerxst ----", 1: "From: (Guy Kuo)Subject: SI Clock Poll - Final CallSummary: Final call for SI clock reportsKeywords: SI,acceleration,clock,upgradeArticle-I.D.: shelley.1qvfo9INNc3sOrganization: University of WashingtonLines: 11NNTP-Posting-Host: carson.u.washington.eduA fair number of brave souls who upgraded their SI clock 

### Tokenize
- Create **sent_to_words()** 
    - Use **gensim.utils.simple_preprocess**
    - Use **generator** instead of an usual function

In [269]:
def sent_to_words(data):
    data_tokens = gensim.utils.simple_preprocess(doc=data, deacc=False, min_len=2, max_len=15)
    yield data_tokens

In [270]:
data_tokens = next(sent_to_words(data))

### Stop words Removal
- Extend the stop words corpus with the following words
    - from
    - subject
    - re
    - edu
    - use

In [271]:
en_stop = stopwords.words('english')
en_stop.append(['from', 'subject', 're', 'edu', 'use'])

#### remove_stopwords( )

In [272]:
def remove_stopwords(texts):
    stopped_tokens = [raw for raw in texts if not raw in en_stop]
    return stopped_tokens

In [273]:
data_tokens = remove_stopwords(data_tokens)
data_tokens

['wheres',
 'thing',
 'subject',
 'car',
 'nntp',
 'posting',
 'host',
 'rac',
 'wam',
 'umd',
 'eduorganization',
 'university',
 'maryland',
 'college',
 'parklines',
 'wondering',
 'anyone',
 'could',
 'enlighten',
 'car',
 'sawthe',
 'day',
 'door',
 'sports',
 'car',
 'looked',
 'late',
 'early',
 'called',
 'bricklin',
 'doors',
 'really',
 'small',
 'addition',
 'front',
 'bumper',
 'separate',
 'rest',
 'body',
 'know',
 'anyone',
 'tellme',
 'model',
 'name',
 'engine',
 'specs',
 'yearsof',
 'production',
 'car',
 'made',
 'history',
 'whatever',
 'info',
 'youhave',
 'funky',
 'looking',
 'car',
 'please',
 'mail',
 'thanks',
 'il',
 'brought',
 'neighborhood',
 'lerxst',
 'guy',
 'kuo',
 'subject',
 'si',
 'clock',
 'poll',
 'final',
 'callsummary',
 'final',
 'call',
 'si',
 'clock',
 'reportskeywords',
 'si',
 'acceleration',
 'clock',
 'upgradearticle',
 'shelley',
 'qvfo',
 'innc',
 'sorganization',
 'university',
 'washingtonlines',
 'nntp',
 'posting',
 'host',
 'cars

### Lemmatization
- Use spacy
    - Download spacy en model (if you have not done that before)
    - Load the spacy model

In [274]:
#! python -m spacy download en

In [275]:
import spacy

In [276]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

#### lemmatizaton( )

In [277]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [278]:
lemmatizer = WordNetLemmatizer()

In [279]:
# lemmatize tokens
lemma_tokens = [lemmatizer.lemmatize(tokens) for tokens in data_tokens]
    
# remove word containing only single char
new_lemma_tokens = [raw for raw in lemma_tokens if not len(raw) == 1]
    

In [280]:
#data_lemmatized = lemmatization(data_tokens, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [281]:
#print(data_lemmatized)

In [282]:
print(new_lemma_tokens)

['wheres', 'thing', 'subject', 'car', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'eduorganization', 'university', 'maryland', 'college', 'parklines', 'wondering', 'anyone', 'could', 'enlighten', 'car', 'sawthe', 'day', 'door', 'sport', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'door', 'really', 'small', 'addition', 'front', 'bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'yearsof', 'production', 'car', 'made', 'history', 'whatever', 'info', 'youhave', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst', 'guy', 'kuo', 'subject', 'si', 'clock', 'poll', 'final', 'callsummary', 'final', 'call', 'si', 'clock', 'reportskeywords', 'si', 'acceleration', 'clock', 'upgradearticle', 'shelley', 'qvfo', 'innc', 'sorganization', 'university', 'washingtonlines', 'nntp', 'posting', 'host', 'carson', 'washington', 'edua', 'fair', 'number', 'brave', 'soul', 'upgraded', 'si', 'clock', 'osc

### Create a Dictionary

In [283]:
dictionary = Dictionary([new_lemma_tokens])

### Filter low-frequency words

In [284]:
#dictionary.filter_extremes(no_below=10, no_above=0.5)

### Create Corpus

In [285]:
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in [new_lemma_tokens]]

In [286]:
corpus

[[(0, 2),
  (1, 1),
  (2, 1),
  (3, 2),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 5),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 2),
  (16, 1),
  (17, 1),
  (18, 5),
  (19, 1),
  (20, 7),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 3),
  (27, 2),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 4),
  (33, 1),
  (34, 3),
  (35, 6),
  (36, 1),
  (37, 2),
  (38, 1),
  (39, 1),
  (40, 3),
  (41, 4),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 10),
  (48, 3),
  (49, 1),
  (50, 3),
  (51, 1),
  (52, 3),
  (53, 2),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 2),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 2),
  (64, 3),
  (65, 2),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 7),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 3),
  (78, 1),
  (79, 5),
  (80, 2),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 2),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 2),
  (91, 2

### Create Index 2 word dictionary

In [287]:
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

In [288]:
id2word

{0: 'ab',
 1: 'abiding',
 2: 'abig',
 3: 'able',
 4: 'ablow',
 5: 'aboard',
 6: 'aboutcryptology',
 7: 'aboutjesus',
 8: 'abouttheir',
 9: 'abouttto',
 10: 'abraham',
 11: 'absolute',
 12: 'absurd',
 13: 'abuse',
 14: 'accel',
 15: 'acceleration',
 16: 'acceptance',
 17: 'accepted',
 18: 'access',
 19: 'accessible',
 20: 'accident',
 21: 'accidental',
 22: 'accidentally',
 23: 'accomplish',
 24: 'account',
 25: 'achieve',
 26: 'achieved',
 27: 'acne',
 28: 'acnefrom',
 29: 'acouple',
 30: 'acquired',
 31: 'acquisition',
 32: 'acrv',
 33: 'acsu',
 34: 'act',
 35: 'action',
 36: 'actionsare',
 37: 'active',
 38: 'activity',
 39: 'actrix',
 40: 'actual',
 41: 'actually',
 42: 'actuallyfaster',
 43: 'adam',
 44: 'adamantly',
 45: 'adapter',
 46: 'adc',
 47: 'add',
 48: 'added',
 49: 'adding',
 50: 'addition',
 51: 'additional',
 52: 'address',
 53: 'adequately',
 54: 'adifferent',
 55: 'administration',
 56: 'adminreply',
 57: 'admission',
 58: 'admit',
 59: 'admitted',
 60: 'adoctors',
 6

### Build a News Topic Model

#### LdaModel
- **num_topics** : this is the number of topics you need to define beforehand
- **chunksize** : the number of documents to be used in each training chunk
- **alpha** : this is the hyperparameters that affect the sparsity of the topics
- **passess** : total number of training assess

In [289]:
ldamodel = LdaModel(corpus, num_topics=15, id2word = id2word, passes=20)

/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.7/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

### Print the Keyword in the 10 topics

In [290]:
pprint(ldamodel.top_topics(corpus,topn=10))

[([(0.00028325015, 'would'),
   (0.00028251312, 'subject'),
   (0.00028163972, 'one'),
   (0.00028084454, 'get'),
   (0.00028068665, 'year'),
   (0.00028063302, 'organization'),
   (0.00028061544, 'dont'),
   (0.00028050237, 'article'),
   (0.00028046584, 'host'),
   (0.000280204, 'car')],
  1.0000889005818412e-12),
 ([(0.00028188984, 'would'),
   (0.0002818189, 'subject'),
   (0.00028133925, 'one'),
   (0.00028105464, 'year'),
   (0.0002807469, 'think'),
   (0.00028066814, 'writes'),
   (0.00028031363, 'get'),
   (0.00028029838, 'dont'),
   (0.00028017853, 'article'),
   (0.00028001034, 'organization')],
  1.0000889005818412e-12),
 ([(0.00028635212, 'subject'),
   (0.00028419704, 'would'),
   (0.00028299884, 'year'),
   (0.00028262284, 'one'),
   (0.00028193256, 'car'),
   (0.00028193125, 'get'),
   (0.0002816419, 'think'),
   (0.0002815489, 'writes'),
   (0.00028140587, 'article'),
   (0.00028121154, 'thing')],
  1.0000889005818412e-12),
 ([(0.0002843439, 'subject'),
   (0.0002826759

## Evaluation of Topic Models
- Model Perplexity
- Topic Coherence

### Model Perplexity

Model perplexity is a measurement of **how well** a **probability distribution** or probability model **predicts a sample**

In [291]:
import collections

In [292]:
def unigram(tokens):    
    model = collections.defaultdict(lambda: 0.01)
    for f in tokens:
        try:
            model[f] += 1
        except KeyError:
            model [f] = 1
            continue
    N = float(sum(model.values()))
    for word in model:
        model[word] = model[word]/N
    return model

In [293]:
model = unigram(new_lemma_tokens)

In [294]:
model['thing']

0.003006496336106244

In [295]:
def perplexity(testset, model):
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
    perplexity = pow(perplexity, 1/float(N)) 
    return perplexity

In [296]:
testset = new_lemma_tokens

In [297]:
perplexity(testset, model)

inf

### Topic Coherence
Topic Coherence measures score a single topic by measuring the **degree of semantic similarity** between **high scoring words** in the topic.

In [298]:
from gensim.models import CoherenceModel

In [299]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=[new_lemma_tokens], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

In [300]:
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3164081567591058


### Visualize the Topic Model
- Use **pyLDAvis**
    - designed to help users **interpret the topics** in a topic model that has been fit to a corpus of text data
    - extracts information from a fitted LDA topic model to inform an interactive web-based visualization

In [301]:
import pyLDAvis.gensim_models

In [302]:
pyLDAvis.enable_notebook()

In [303]:
pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary, mds='mmds')

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:58: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted t

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.078165  0.030596       1        1  99.988261
0     -0.005587 -0.002188       2        1   0.000838
1     -0.005587 -0.002187       3        1   0.000838
2     -0.005584 -0.002185       4        1   0.000838
3     -0.005585 -0.002187       5        1   0.000838
4     -0.005578 -0.002184       6        1   0.000838
5     -0.005578 -0.002182       7        1   0.000838
6     -0.005581 -0.002185       8        1   0.000838
8     -0.005587 -0.002187       9        1   0.000838
9     -0.005589 -0.002187      10        1   0.000838
10    -0.005581 -0.002185      11        1   0.000838
11    -0.005586 -0.002186      12        1   0.000838
12    -0.005579 -0.002184      13        1   0.000838
13    -0.005583 -0.002185      14        1   0.000838
14    -0.005581 -0.002185      15        1   0.000838, topic_info=         Term       Freq      Total Category  logprob  loglift
2978  subject  57.000000  57.000000  Default  30.0000  30.0000
3544    would  42.000000  42.000000  Default  29.0000  29.0000
2169      one  40.000000  40.000000  Default  28.0000  28.0000
3570     year  37.000000  37.000000  Default  27.0000  27.0000
217   article  35.000000  35.000000  Default  26.0000  26.0000
...       ...        ...        ...      ...      ...      ...
3352      use   0.000019  17.534735  Topic15  -8.1810  -2.0643
493       car   0.000019  28.210412  Topic15  -8.1813  -2.5400
3173     time   0.000019  17.534440  Topic15  -8.1813  -2.0646
1784     line   0.000019  16.564171  Topic15  -8.1814  -2.0077
2121   number   0.000019  13.652305  Topic15  -8.1818  -1.8148

[949 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
8         1  0.965702     abouttheir
25        1  0.965710        achieve
36        1  0.965697     actionsare
43        1  0.965681           adam
46        1  0.965710            adc
...     ...       ...            ...
3551      1  0.998089         writes
3558      1  0.965687  x_shmputimage
3570      1  1.002218           year
3589      1  0.965704    yourmessage
3596      1  0.965701           zero

[488 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15])